In [50]:
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [206]:
# function for calculating the discrete fourier transform 
# FULL DISCOLSURE: I have seen code for the DFT and FFT and 
# have used some as references. 
# Coding this up on my own for practice mostly...

def dft_1d(x):
    """
    A discrete fourier transform algorithm in 1D
    
    x: ndarray of size (N)
    """
    # x is a 1D vector for which we calculate a DFT
    x = np.asarray(x,dtype=float)
    N = x.shape[0]
    # Evaluate xn on n points between 0 and N
    n = np.arange(N)
    # make an orthogonal vector to n
    k = n.reshape((N,1))
    # calculate conversion (k * n produces an NxN matrix)
    W = np.exp(-2j*np.pi*k*n/N)

    # Return row-wise sum over the conversion
    return np.dot(W,x)
    
    
    
def fft_1d(f,diagnostics=False):
    """
    A fast fourier transform algorithm in 1D. Heavily based off of some 
    clever code I found online (at least the setup of the while loop).
    
    Apparently this method using vectors is faster than recursion, which 
    I think may have something to do with the computational cost 
    of stacking versus calling/reasigning array values (?).
    
    f: ndarray of size (N)
    """
    N = f.shape[0]
    if np.log2(N) % 1 > 0:
        raise ValueError("the length of x must be a factor of 2")    
    # bottom should be size 2
    min_N = min(N,2)
    # points to iterate over
    n = np.arange(min_N)
    # add an extra dimension for summing
    k = n[:,None]
    # Compute DFT of the lowest level (2 points), 
    # as we did above
    W = np.exp(-2j*np.pi*k*n/min_N)
    # Start with the bottom DFT, done by collapsing 
    # array into [evens,odds] and dotting with 2-point DFT
    F = np.dot(W, f.reshape((min_N,-1)))
    # while loop
    while F.shape[0] < N:
        if diagnostics==True:
            # Watch the shape of X evolve over the while loop
            print(F.shape)
        # Subdivide X ()
        F_even = F[:,:int(F.shape[1]/2)]
        F_odd = F[:,int(F.shape[1]/2):]
        # Compute coefficient for odd sums
        N_F = F.shape[0]
        n_F = np.arange(N_F)
        Cn = np.exp(-1j*np.pi*n_F/N_F)[:,None]
        F = np.vstack([F_even + Cn*F_odd,
                       F_even - Cn*F_odd])

    # perform one final flattening, return fourier-transformed data
    return F.ravel()


def ifft_1d(F,diagnostics=False):
    """
    A fast inverse fourier transform algorithm in 1D. Based on my
    stolen fft code above, since the concept seems similar enough
    
    F: ndarray of size (K), switched cases to keep things aesthetic
    """
    K = F.shape[0]
    if np.log2(K) % 1 > 0:
        raise ValueError("the length of F must be a factor of 2")    
    min_K = min(K,2)
    k = np.arange(min_K)
    n = k[:,None]
    w = np.exp(2j*np.pi*k*n/min_K)
    f = np.dot(w, F.reshape((min_K,-1)))
    while f.shape[0] < K:
        if diagnostics==True:
            print(f.shape)
        f_even = f[:,:int(f.shape[1]/2)]
        f_odd = f[:,int(f.shape[1]/2):]
        K_f = f.shape[0]
        k_f = np.arange(K_f)
        Ck = np.exp(1j*np.pi*k_f/K_f)[:,None]
        f = np.vstack([f_even + Ck*f_odd,
                       f_even - Ck*f_odd])

    return f.ravel()


def fft_2d(a):
    """
    Code is entirely based on what Professor Hoskins said in class:
    "Do FFT of each column, transpose, and then do FFT again," 
    which sounds pretty easy
    
    a: ndarray of size (N,N). It's lowercase because its our function..
    """    
    # initialize by doing fft on the first column
    A0 = fft_1d(a[:,0])
    # loop through the rest...
    for n in range(1,a.shape[1]):
        # vstacking transposes the matrix ¯\_(ツ)_/¯ 
        A0 = np.vstack((A0,fft_1d(a[:,n])))
    # initialize by doing fft on the first column
    A1 = fft_1d(A0[:,0])
    # loop through the rest...
    for n in range(1,A0.shape[1]):
        A1 = np.vstack((A1,fft_1d(A0[:,n])))
        
    return A1

In [ ]:
# Verify efficacy for 1D functions using numpy 
x = np.random.random(1024)
print("My 1d fft works!")
print("dft_1d(x) == np.fft.fft(x)",np.allclose(dft_1d(x), np.fft.fft(x)))
print("fft_1d(x) == np.fft.fft(x)",np.allclose(fft_1d(x), np.fft.fft(x)))
print("time trials:")
print("My function:")
%timeit dft_1d(x)
print("Numpy implementation:")
%timeit np.fft.fft(x)
print("\n\n")

# Verify efficacy for 2D functions using numpy 
x2d = np.random.random((1024,1024))

print("My 2d fft works!")
print("fft_2d(xd2) == np.fft.fft2(x2d)",np.allclose(fft_2d(x2d), np.fft.fft2(x2d)))
print("time trials:")
print("My function:")
%timeit fft_2d(x2d)
print("Numpy implementation:")
%timeit np.fft.fft2(x2d
print("numpy is much faster (roughly 2N times faster in my calculation...)")


In [223]:
def convolve_2d_circulant(A,f):
    """
    2D convolutions using fft
    
    A: ndarray of size (N,N) containing transformed operator
    f: ndarray of size (N,N) containing target function
    """ 
    F = fft_2d(f) 
    
    return np.fft.ifft2(A*F)
    
    
a = np.zeros((4,4))
f = np.zeros((4,4))
# make circulant matrix
for i in range(-a.shape[0]+1,0):
    a += np.eye(4,k=i)*(4+i)
    a += np.eye(4,k=-i)*(-i)
for i in range(0,f.shape[0]):
    for j in range(0,f.shape[0]):
        f[i,j] = i + j**2
print("Circulant operator matrix")
print(a)
print("Function matrix:")
print(f)

A = fft_2d(a)
Af_circulant = convolve_2d_circulant(A,f)
# test against pure numpy implementation (should be close to the same)
h = np.fft.ifft2(np.fft.fft2(a)*np.fft.fft2(f))
print("\n\nOutput:")
print(Af_circulant)
print("\nvanilla numpy output:")
print(h)
print("\nAppears to give us good precision. Mean error:",np.mean(Af_circulant-h))
print("time trials:")
print("My function:")
%timeit convolve_2d_circulant(A,f)
print("Pure numpy:")
%timeit np.fft.ifft2(np.fft.fft2(a)*np.fft.fft2(f))
print("\nnumpy is still much faster (order 10)")

Circulant operator matrix
[[0. 1. 2. 3.]
 [3. 0. 1. 2.]
 [2. 3. 0. 1.]
 [1. 2. 3. 0.]]
Function matrix:
[[ 0.  1.  4.  9.]
 [ 1.  2.  5. 10.]
 [ 2.  3.  6. 11.]
 [ 3.  4.  7. 12.]]


Output:
[[120.-2.958635600798e-15j 120.+2.031490615409e-15j
  120.+3.180680205723e-15j 120.-1.809446010484e-15j]
 [120.+2.684018541945e-15j 120.+4.473981269584e-15j
  120.+6.466505319303e-16j 120.-1.143312195709e-15j]
 [120.+1.948328284476e-17j 120.+6.785966338452e-15j
  120.-2.415278877698e-16j 120.-7.008010943377e-15j]
 [120.-5.623170859899e-15j 120.+4.343475684277e-15j
  120.+2.292501786023e-15j 120.-7.674144758152e-15j]]

vanilla numpy output:
[[120.+0.j 120.+0.j 120.+0.j 120.+0.j]
 [120.+0.j 120.+0.j 120.+0.j 120.+0.j]
 [120.+0.j 120.+0.j 120.+0.j 120.+0.j]
 [120.+0.j 120.+0.j 120.+0.j 120.+0.j]]

Appears to give us good precision. Mean error: 0j
370 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
33.8 µs ± 629 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

numpy is stil

In [227]:
def convolve_2d_toep(A,f):
    """
    2D convolutions using fft for a non-circulant operator
    
    A: ndarray of size (N,N) containing transformed operator
    f: ndarray of size (N,N) containing target function
    """ 
    
    # add padding to f to make it shape 2Nx2N (adding an extra zero row/column
    # to make my fft_2d code work faster)
    f_pad = np.zeros((2*f.shape[0]-1,2*f.shape[0]-1))
    f_pad[:f.shape[0],:f.shape[0]] = f
    F_pad = np.fft.fft2(f_pad)
    
    # return an NxN matrix fir Af
    return np.fft.ifft2(A*F_pad)#[:f.shape[0],:f.shape[0]]


########################################################
# initialization
a = np.zeros((4,4))
f = np.zeros((4,4))
for i in range(0,4):
    for j in range(0,4):
        a[i,j] = i - j
        f[i,j] = i + j**2
a_toep = np.zeros((2*a.shape[0]-1,2*a.shape[0]-1))
for i in range(0,a.shape[0]):
    a_toep += np.eye(2*a.shape[0]-1,k=i)*a[0,i]
    a_toep += np.eye(2*a.shape[0]-1,k=-i-3)*a[0,-i]
    a_toep += np.eye(2*a.shape[0]-1,k=-i)*a[i,0]
    a_toep += np.eye(2*a.shape[0]-1,k=i+3)*a[-i,0]
print("Toeplitz matrix:")
print(a_toep)
# precompute A_toep
A_toep = np.fft.fft2(a_toep)

########################################################
# Output
Af_toep = convolve_2d_toep(A_toep,f)[:4,:4]
h = signal.fftconvolve(a_toep,f)[3:7,3:7]
print("\nMy output")
print(Af_toep)
print("\nscipy.signal.fftconvolve output:")
print(h)
print("\nAppears to give us 16 digits. Mean error:",np.mean(Af_toep-h))
print("time trials:")
print("My function:")
%timeit convolve_2d_toep(A_toep,f)[:4,:4]
print("scipy.signal.fftconvolve:")
%timeit signal.fftconvolve(a_toep,f)[3:7,3:7]
print("\nMine is much faster because I precomputed A_toep...")

Toeplitz matrix:
[[ 0. -1. -2. -3.  3.  2.  1.]
 [ 1.  0. -1. -2. -3.  3.  2.]
 [ 2.  1.  0. -1. -2. -3.  3.]
 [ 3.  2.  1.  0. -1. -2. -3.]
 [-3.  3.  2.  1.  0. -1. -2.]
 [-2. -3.  3.  2.  1.  0. -1.]
 [-1. -2. -3.  3.  2.  1.  0.]]

My output
[[ 4.00000000000e+01+0.j -1.90000000000e+01+0.j -5.70000000000e+01+0.j
  -6.00000000000e+01+0.j]
 [ 5.70000000000e+01+0.j  4.00000000000e+01+0.j -1.90000000000e+01+0.j
  -5.70000000000e+01+0.j]
 [ 3.90000000000e+01+0.j  5.70000000000e+01+0.j  4.00000000000e+01+0.j
  -1.90000000000e+01+0.j]
 [ 1.06581410364e-14+0.j  3.90000000000e+01+0.j  5.70000000000e+01+0.j
   4.00000000000e+01+0.j]]

scipy.signal.fftconvolve output:
[[ 40. -19. -57. -60.]
 [ 57.  40. -19. -57.]
 [ 39.  57.  40. -19.]
 [  0.  39.  57.  40.]]

Appears to give us 16 digits. Mean error: (2.220446049250313e-16+0j)
time trials:
My function:
27.2 µs ± 734 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
scipy.signal.fftconvolve:
418 µs ± 3.95 µs per loop (mean ± std. dev.